# 0. Import Modules and setup parameters

In [34]:
import cv2, os
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
print(os.getcwd())

C:\Users\Hai\Documents\UQ\UQ 2022 - Sixth Year\METR4911_ML_Python\Object_Detection_Tutorial_YouTube\TFODCourse


In [27]:
# coco_labels.txt, Labels for fruit/vegetable (6 labels) 
labels = {0: "Person", 51: "Banana", 52: "Apple", 54: "Orange", 55: "Broccoli", 56: "Carrot"}

# Color in BGR in CV2 not RGB, match color with object's color
colors = {"Person": (179,179,255), "Banana": (53, 225,255), "Apple": (0,8,255), 
         "Carrot": (33,145,237), "Orange": (0,126,255), "Broccoli": (0,255,74)}

# Input Resize Image
img_width, img_height  = (640, 480)

# 1. Loading Mask RCNN

In [28]:
# Get file path of weights(.pb) and config (.pbtxt)
frozen_pb = os.path.join("cv_dnn", "mask_rcnn_inception_v2_coco_2018_01_28", "frozen_inference_graph.pb")
pb_txt = os.path.join("cv_dnn", "mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")

print("Weights:", frozen_pb)
print("Config:",pb_txt)

# Load the weights and the config of Mask RCNN
net = cv2.dnn.readNetFromTensorflow(frozen_pb, pb_txt)

Weights: cv_dnn\mask_rcnn_inception_v2_coco_2018_01_28\frozen_inference_graph.pb
Config: cv_dnn\mask_rcnn_inception_v2_coco_2018_01_28.pbtxt


## 1.1 Object Detection and Mask Segmentation Function

In [29]:
def object_detection_mask(img):
    height, width, _ = img.shape  # (row, column)
    
    ########################### Generate random colors and create black image ###########################
    black_image = np.zeros((height, width, 3), np.uint8)
    
    ########################### Detect objects ###########################
    blob = cv2.dnn.blobFromImage(img, swapRB=True)  # Get blob of input image
    net.setInput(blob)
    
    # boxes.shape (1,1,100,7),  mask.shape (100,90,15,15)
    boxes, masks = net.forward(["detection_out_final", "detection_masks"])  # Actual prediction (takes time)
    detection_count = boxes.shape[2]  # number of objects detected, 100 is the limit
    
    ############################ For each objects detected, draw box and mask ###########################
    for i in range(detection_count):
        # box [0,label,confidence,x1,y1,x2,y2] coods are normalised to 0-1 so multiply 
        #     the coods with image's width and height
        box = boxes[0, 0, i]
        class_id = int(box[1])  # Label/Class number, refer to coco_labels.txt
        score = box[2]          # Confidence percentage
        
        # If score below (is part of label) or not part of labels want to detect skip
        if class_id not in labels or score < 0.5:
            continue

        color = colors[labels[class_id]] # Color of object, defined in section 0
        
        ############################ Get Box Coordinates and Draw Box ############################
        x = int(box[3] * width)    # Top left
        y = int(box[4] * height)   # Top left
        x2 = int(box[5] * width)   # Bottom right
        y2 = int(box[6] * height)  # Bottom right
        cv2.rectangle(img, (x,y), (x2,y2), (int(color[0]), int(color[1]), int(color[2])) , 2) # Draw rectangle box

        ############################ Get the mask ############################
        roi = black_image[y: y2, x: x2]
        roi_height, roi_width, _ = roi.shape
        mask = masks[i, int(class_id)]  # mask.shape -> (15, 15)
        mask = cv2.resize(mask, (roi_width, roi_height))
        
        # 0.5 is threshold of confidence score of each pixel? so over 0.5 becomes 255 and 0 otherwise
        _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)
        
        ############################ Get Mask Coordinates ############################
        # Extract boundary of object
        contours, _ = cv2.findContours(np.array(mask, np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)         
        
        ############################ Draw and Fill Polygons ############################
        for cnt in contours:
            cv2.fillPoly(roi, [cnt], (int(color[0]), int(color[1]), int(color[2])))
    
    ############################ Display output image on computer ############################
    cv2.imshow("Frame", img)
    cv2.imshow("Black Image", black_image)

In [30]:
# Have function for drawing rectangle box and add text label in, (x1,y1,x2,y2,color,label,padding,confidence)

# 2. Detect Static Image

In [31]:
# Load image
img_file = "banana.d8fbcaf6-03f4-11ed-a1f3-95c0acd580f9.jpg"
img_path = os.path.join("old_test_train_images_xml", "Fruit_Vegetable_15-July-22", "collectedimages", "banana", img_file)

print(img_path)

old_test_train_images_xml\Fruit_Vegetable_15-July-22\collectedimages\banana\banana.d8fbcaf6-03f4-11ed-a1f3-95c0acd580f9.jpg


In [32]:
# Read image and resize image to (640, 480) or other dimensions
img = cv2.imread(img_path)
img = cv2.resize(img, (img_width,img_height), interpolation = cv2.INTER_AREA)
height, width, _ = img.shape

# 0 = open until any key is pressed, then close, any other number, in milliseconds, opens window in that duration
cv2.imshow("Image ({},{})".format(width, height), img)
cv2.waitKey(0)  
cv2.destroyAllWindows()

print("Width:", width)
print("Height:", height)

Width: 640
Height: 480


In [33]:
# For Static Image Only
object_detection_mask(img)

cv2.waitKey(0)
cv2.destroyAllWindows() 

# 3. Real Time Video Object Detection and Segmentation

In [25]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    object_detection_mask(frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [18]:
# Debuggin, or webcam still on when having errors
cap.release()
cv2.destroyAllWindows()

(179, 179, 255)
